# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

To create the database: 
1. Navigate to the correct folder on your local drive in your terminal. Type in: 
mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

This creates a database named uk_food and a collection named establishments. 



In [21]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint



In [22]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [23]:
# confirm that our new database was created
print(mongo.list_database_names())


['admin', 'autosaurus', 'classDB', 'config', 'fruits_db', 'garden_db', 'local', 'met', 'pesitly_marketing', 'petsitly_marketing', 'travel_db', 'uk_food']


In [24]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [25]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [26]:
# review a document in the establishments collection
pprint(db.establishments.find_one())


{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63d6b8e3285f68692832bb26'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None

In [27]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [28]:
# Create a dictionary for the new restaurant data
#Because I do not have any other data on the restaurant, I am only adding in fields for which I have data
penang_flavors = {
 'BusinessName': 'Penang Flavours',
 'AddressLine4': 'Greenwich',
   }



In [29]:
# Insert the new restaurant into the collection
establishments.insert_one(penang_flavors)

In [30]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({'BusinessName':'Penang Flavours'}))

{'AddressLine4': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 '_id': ObjectId('63d6b9116d8e788fcf334368'),
 'geocode': {'latitude': None, 'longitude': None}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [31]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

#the query variable defines the field : value I am looking for
query = {'BusinessType' : 'Restaurant/Cafe/Canteen'}

#the fields variable flags the fields as TRUE to return in the query
fields = {'BusinessType': 1, 'BusinessTypeID': 1}

#results uses the query and fields variables to run the query and return the fields I am interested in
results = db.establishments.find(query, fields)

pprint(results[0])


{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('63d6b8e3285f68692832bb26')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [32]:
# Update the new restaurant with the correct BusinessTypeID

#create a variable for finding the restaurant
query = {'BusinessName': 'Penang Flavours'}

#create a variable for adding the field to the record
fields = {'$set': {'BusinessTypeID': 1}}

#use update_one to update the restaurant's record
update = db.establishments.update_one(query, fields)


In [33]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
result = db.establishments.find_one(query)

pprint(result)

{'AddressLine4': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 '_id': ObjectId('63d6b9116d8e788fcf334368'),
 'geocode': {'latitude': None, 'longitude': None}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [34]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}

print("Number of documents with LocalAuthorityName as 'Dover':", establishments.count_documents(query))

Number of documents with LocalAuthorityName as 'Dover': 0


In [35]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName': 'Dover'}

delete = db.establishments.delete_many(query)


In [36]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': 'Dover'}

print("Number of documents with LocalAuthorityName as 'Dover':", establishments.count_documents(query))

Number of documents with LocalAuthorityName as 'Dover': 0


In [37]:
# Check that other documents remain with 'find_one'
fields = {'LocalAuthorityName':1}
query = {}

results = db.establishments.find_one(query, fields)

pprint(results)

{'LocalAuthorityName': 'Folkestone and Hythe',
 '_id': ObjectId('63d6b8e3285f68692832bb26')}


5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [38]:
# Change the data type from String to Decimal for longitude
query = {}
type_change = [{'$set':{ "geocode.longitude" : {'$toDouble': "$geocode.longitude"}}}]
update = db.establishments.update_many(query, type_change)

#verify
fields = {'geocode.latitude':1, 'geocode.longitude':1}
pprint(db.establishments.find_one(query, fields))

{'_id': ObjectId('63d6b8e3285f68692832bb26'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625}}


In [39]:
# Change the data type from String to Decimal for latitude
query = {}
type_change = [{'$set':{ "geocode.latitude" : {'$toDouble': "$geocode.latitude"}}}]
update = db.establishments.update_many(query, type_change)

#verify
fields = {'geocode.latitude':1, 'geocode.longitude':1}
pprint(db.establishments.find_one(query, fields))

{'_id': ObjectId('63d6b8e3285f68692832bb26'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625}}


In [40]:
# Check that the coordinates are now numbers
query = {}
fields = {'geocode.latitude':1, 'geocode.longitude':1}
results = db.establishments.find(query, fields)

pprint(list(results[0:10]))


[{'_id': ObjectId('63d6b8e3285f68692832bb26'),
  'geocode': {'latitude': 51.083812, 'longitude': 1.195625}},
 {'_id': ObjectId('63d6b8e3285f68692832bb27'),
  'geocode': {'latitude': 51.086058, 'longitude': 1.196408}},
 {'_id': ObjectId('63d6b8e3285f68692832bb28'),
  'geocode': {'latitude': 51.085797, 'longitude': 1.194762}},
 {'_id': ObjectId('63d6b8e3285f68692832bb2b'),
  'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'_id': ObjectId('63d6b8e3285f68692832bb2c'),
  'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'_id': ObjectId('63d6b8e3285f68692832bb2d'),
  'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'_id': ObjectId('63d6b8e3285f68692832bb2e'),
  'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'_id': ObjectId('63d6b8e3285f68692832bb2f'),
  'geocode': {'latitude': 51.0783519967076, 'longitude': 1.18590330311705}},
 {'_id': ObjectId('63d6b8e3285f68692832bb30'),
  'geocode': {'latitude': 51.0783519967076, 'longitude': 1.18590330311